In [1]:
import tensorflow as tf

### tensorflow中有两种队列：
+ FIFOQueue
+ RandomShuffleQueue


>_FIFOQueue_ 创建一个先入先出队列

我们在训练一些语音、文字样本时，使用循环神经网络的网络结构，希望读入的训练样本是有序的，就要用 _FIFOQueue_。

>_RandomShuffleQueue_ 创建一个随机队列，在出队列时，是以随机的顺序产生元素的

我们在训练一些图像样本时，使用 CNN 的网络结构，希望可以无序地读入训练样本，就要用 _RandomShuffleQueue_，每次随机产生一个训练样本。

In [2]:
q = tf.FIFOQueue(3,'float')
init=q.enqueue_many(([.1,.2,.3],))
x=q.dequeue()       #出队
y=x+1               #+1
q_inc=q.enqueue([y])#入队

In [3]:
with tf.Session() as sess:
    sess.run(init)
    quelen = sess.run(q.size())
    for i in range(2):
        sess.run(q_inc)
    quelen = sess.run(q.size())
    for i in range(quelen):
        print(sess.run(q.dequeue()))

0.3
1.1
1.2


RandomShuffleQueue 在 TensorFlow 使用异步计算时非常重要。因为 TensorFlow 的会话是 支持多线程的，我们可以在主线程里执行训练操作，使用 RandomShuffleQueue 作为训练输入， 开多个线程来准备训练样本，将样本压入队列后，主线程会从队列中每次取出 mini-batch 的样 本进行训练

In [4]:
q = tf.RandomShuffleQueue(capacity=10,min_after_dequeue=2,dtypes='float')

In [5]:
sess = tf.Session()
for i in range(0,10):
    sess.run(q.enqueue(i))
for i in range(0,8):
    print(sess.run(q.dequeue()))

6.0
2.0
7.0
0.0
4.0
8.0
5.0
1.0


#### 阻断现象
+ 队列长度等于最小值，执行出队操作
+ 队列长度等于最大值，执行入队操作

只有队列满足要求后才能继续执行，可以通过设置session在运行时的等待时间来解除阻断

In [6]:
run_options=tf.RunOptions(timeout_in_ms=10000)
try:
    sess.run(q.dequeue(),options=run_options)
except tf.errors.DeadlineExceededError:
    print('out of range')

out of range
